In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set("spark.sql.execution.pyarrow.enabled", "true")
        .set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.4.2")
        .set('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.spark_catalog.type', 'hive')
        .set('spark.sql.catalog.local', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.local.type', 'hadoop')
        .set('spark.sql.catalog.local.warehouse', '/warehouse/')
        .set("spark.sql.catalog.defaultCatalog", 'local')
        .set("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
)

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-036495d9-6b2b-4fa9-bfcb-c066fe49677f;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 in central
:: resolution report :: resolve 106ms :: artifacts dl 7ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-subm

In [4]:
spark

# Prepare 

In [5]:
spark.sql("USE local;");

In [6]:
spark.sql("""
CREATE TABLE IF NOT EXISTS taxis.trips(
    vendor_id INT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count INT,
    pickup_location_id INT,
    dropoff_location_id INT,
    fare_amount FLOAT
)
USING iceberg;
""").toPandas()

""


# Ingest first batch 

In [7]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trips_table USING csv
OPTIONS (path "/data/yellow_tripdata_sample_2019_01.csv", header true);
""").toPandas()

""


In [8]:
spark.sql("""
SELECT * FROM trips_table LIMIT 5;
""").toPandas()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_location_id,dropoff_location_id,fare_amount
0,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,230,48,6.5
1,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,112,112,6.0
2,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,107,4,6.0
3,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,143,158,11.0
4,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,246,90,6.5


In [9]:
spark.sql("""
INSERT INTO taxis.trips
SELECT 
    CAST(vendor_id AS INT),
    CAST(pickup_datetime AS TIMESTAMP),
    CAST(dropoff_datetime AS TIMESTAMP),
    CAST(passenger_count AS INT),
    CAST(pickup_location_id AS INT),
    CAST(dropoff_location_id AS INT),
    CAST(fare_amount AS FLOAT)
FROM trips_table;
"""
).toPandas()

""


In [10]:
spark.sql("SELECT COUNT(*) FROM taxis.trips;").toPandas()

,count(1)
0,15


# Prepare 2nd batch 

In [11]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trips_table USING csv
OPTIONS (path "/data/yellow_tripdata_sample_2019_02.csv", header true);
""").toPandas()

""


# Merge / Append 

In [12]:
spark.sql("""
WITH changes AS (
    SELECT
        CAST(vendor_id AS INT),
        CAST(pickup_datetime AS TIMESTAMP),
        CAST(dropoff_datetime AS TIMESTAMP),
        CAST(passenger_count AS INT),
        CAST(pickup_location_id AS INT),
        CAST(dropoff_location_id AS INT),
        CAST(fare_amount AS FLOAT)
    FROM trips_table
)

MERGE INTO taxis.trips t
USING changes s
ON
    t.vendor_id           = s.vendor_id AND
    t.pickup_datetime     = s.pickup_datetime AND
    t.dropoff_datetime    = s.dropoff_datetime AND
    t.pickup_location_id  = s.pickup_location_id AND
    t.dropoff_location_id = s.dropoff_location_id
WHEN NOT MATCHED THEN INSERT *
""").toPandas()

""


In [13]:
spark.sql("SELECT COUNT(*) FROM taxis.trips;").toPandas()

,count(1)
0,20


In [14]:
df_main = spark.table("taxis.trips")

In [17]:
all_cols = set(df_main.columns)
skip_cols = set(["passenger_count", "fare_amount"])

In [18]:
salt_cols = list(all_cols - skip_cols)

In [19]:
df_main = (
    df_main
    .withColumn('salted_id', F.sha2(F.concat_ws('_', *salt_cols), 512))
)

In [21]:
df_main.write.saveAsTable("taxis.trips", mode="overwrite")

# Prepare 3rd batch 

In [22]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW trips_table USING csv
OPTIONS (path "/data/yellow_tripdata_sample_2019_02-updates.csv", header true);
""").toPandas()

""


In [24]:
df_changes = spark.table("trips_table")
df_changes = (
    df_changes
    .withColumn('salted_id', F.sha2(F.concat_ws('_', *salt_cols), 512))
)

In [27]:
df_changes.createOrReplaceTempView("trips_table")

In [45]:
spark.sql("SELECT * FROM trips_table;").toPandas()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_location_id,dropoff_location_id,fare_amount,salted_id
0,1,2019-02-07 15:48:06,2019-02-07 16:00:40,2,234,211,9.0,fdfc0fd2719ac98cb78d6e578f11ed150955737a84948b...
1,2,2019-02-11 15:19:56,2019-02-11 15:40:10,1,161,249,14.0,208c8d794ed0befd91f31eedf9ce30370a81b0474048eb...
2,2,2019-02-15 20:03:53,2019-02-15 20:08:34,1,237,161,5.0,62b1191c1503039c299c8f8912ef7868c202d76d4b95f7...
3,2,2019-02-03 15:16:04,2019-02-03 15:21:49,1,100,48,5.5,7f62870b2d7efa5430ab82675105275d37dce91a5eaafd...
4,2,2019-02-15 09:23:09,2019-02-15 09:42:51,6,114,230,13.5,aa813ce880ff719ea63bf99cf10128bdd7c22c7da0448c...


# Merge / Update 

In [50]:
spark.sql("""
WITH changes AS (
    SELECT
        COALESCE(s.salted_id, t.salted_id) AS salted_id,
        CAST(s.vendor_id AS INT),
        CAST(s.pickup_datetime AS TIMESTAMP),
        CAST(s.dropoff_datetime AS TIMESTAMP),
        CAST(s.passenger_count AS INT),
        CAST(s.pickup_location_id AS INT),
        CAST(s.dropoff_location_id AS INT),
        CAST(s.fare_amount AS FLOAT),
        CASE
            WHEN t.salted_id IS NULL THEN 'D'
            WHEN s.salted_id IS NULL THEN 'I'
            ELSE 'U'
        END as cdc
    FROM trips_table AS s
    FULL OUTER JOIN taxis.trips AS t ON s.salted_id = t.salted_id
)
MERGE INTO taxis.trips t
USING changes s
ON
    t.salted_id = s.salted_id
WHEN MATCHED AND s.cdc = 'U' THEN UPDATE SET *
""").toPandas()

""


In [ ]:
# WHEN MATCHED AND changes.cdc = 'D' THEN DELETE
# WHEN MATCHED AND changes.cdc = 'U' THEN UPDATE SET *
# WHEN NOT MATCHED THEN INSERT *

In [51]:
spark.sql("SELECT COUNT(*) FROM taxis.trips;").toPandas()

,count(1)
0,20


In [52]:
spark.sql("SELECT * FROM taxis.trips;").toPandas()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_location_id,dropoff_location_id,fare_amount,salted_id
0,1,2019-02-07 15:48:06,2019-02-07 16:00:40,2,234,211,9.0,fdfc0fd2719ac98cb78d6e578f11ed150955737a84948b...
1,2,2019-02-11 15:19:56,2019-02-11 15:40:10,1,161,249,14.0,208c8d794ed0befd91f31eedf9ce30370a81b0474048eb...
2,2,2019-02-15 20:03:53,2019-02-15 20:08:34,1,237,161,5.0,62b1191c1503039c299c8f8912ef7868c202d76d4b95f7...
3,2,2019-02-03 15:16:04,2019-02-03 15:21:49,1,100,48,5.5,7f62870b2d7efa5430ab82675105275d37dce91a5eaafd...
4,2,2019-02-15 09:23:09,2019-02-15 09:42:51,6,114,230,13.5,aa813ce880ff719ea63bf99cf10128bdd7c22c7da0448c...
5,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,230,48,6.5,73154988e14147baf9ad57b3416d36fea97dcb46ed8760...
6,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,112,112,6.0,e2d1311d59e30c705e96793e9440e612726726edceff54...
7,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,107,4,6.0,71360df112c406ba7ec6abbd23885a619836a95d0edb1c...
8,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,143,158,11.0,0e9c343fa090cbe123ba49bdb1165ed3fab7c00fadf8be...
9,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,246,90,6.5,d392f0f736a286ad4bd79106943a74cf77ed3ae4bd36af...
